In [1]:
%pip -q install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
%pip -q install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

In [4]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
class RecommenderModel(nn.Module):
    def __init__(self, industry_dim, product_dim):
        super(RecommenderModel, self).__init__()
        self.fc_industry = nn.Linear(industry_dim, 128)
        self.fc_product = nn.Linear(product_dim, 64)
        self.fc_out = nn.Linear(64, 1)

        self.lstm = nn.LSTM(64, 64, batch_first=True)

    def forward(self, industry_features, )

SyntaxError: expected ':' (2214949800.py, line 2)